In [18]:
import plotly.plotly as py
from plotly.graph_objs import Scatter
from IPython.display import Image

In [19]:
from plotly.widgets import GraphWidget

In [25]:
url = py.plot({'data': [Scatter(x=[1, 2, 3], y=[4, 6, 9]), Scatter(x=[1, 2, 3], y=[10, 30, 20])]}, 
               filename = 'widget template', auto_open=False)


High five! You successfuly sent some data to your account on plotly. View your plot in your browser at https://plot.ly/~mcian91/0 or inside your plot.ly account where it is named 'widget template'


In [21]:
graph = GraphWidget(url)
g = graph
from IPython.display import Image, display, clear_output
display(graph)

In [22]:
from IPython.display import display, clear_output
def message_handler(widget, msg):
    clear_output()
    print(widget._graph_url)
    display(msg)

g.on_click(message_handler)

In [23]:
%%html
<img src=url>

In [26]:
print(url)
from IPython.display import Image
Image(url=url)

https://plot.ly/~mcian91/0


In [28]:
contour_fig = py.get_figure(url)

In [29]:
def march(x0, y0, x1, y1):
    '''
    Return the closest path of integer coordinates 
    between (x0, y0) and (x1, y1)
    '''
    if abs(x1-x0) > abs(y1-y0):    
        if x1>x0:
            x = range(int(x0), int(x1)+1)
        else:
            x = range(int(x0), int(x1)+1, -1)
        y = []
        tanth = (y1-y0)/(x1-x0)
        for xi in x:
            y.append(round(y0+(xi-x0)*tanth))
    else:
        if y1>y0:
            y = range(int(y0), int(y1)+1)
        else:
            y = range(int(y0), int(y1)+1, -1)
        x = []
        tanth = (x1-x0)/(y1-y0)
        for yi in y:
            x.append(round(x0+(yi-y0)*tanth))

    return (x, y)

class Responder(object):
    '''
    Stateful object that stores and computes the 
    elevation and distance data of the 
    line plot. The 'click' method is executed
    on `click` events of the contour map.
    '''
    def __init__(self, data):
        self.clicks = 0
        self.x = []
        self.y = []
        self.xp = []
        self.yp = []
        self.z = []
        self.d = []
        self.data = data

    def append_point(self, point):
        xp = point['pointNumber'][1]
        yp = point['pointNumber'][0]
        self.xp.append(xp)
        self.yp.append(yp)

        if 'x' in self.data[0]:
            x = self.data[0]['x'][xp]
        else:
            x = xp
        if 'y' in self.data[0]:
            y = self.data[0]['y'][xp]
        else:
            y = yp

        self.x.append(x)
        self.y.append(y)
        self.z.append(point['z'])
    
        if len(self.x) == 1:
            self.d.append(0)
        else:
            self.d.append(math.sqrt((y-self.y[-2])**2+(x-self.x[-2])**2))
        
        
    def click(self, contour_widget, click_obj):
        point = click_obj[0]
        if self.clicks==0 or self.clicks > 5:
            self.__init__(self.data)
            self.append_point(point)
        else:
            
            (xpath, ypath) = march(self.xp[-1], self.yp[-1],
                                   point['pointNumber'][1], point['pointNumber'][0])
    
            d = []
            z = []
            for i in range(1, len(xpath)):
                xpi = xpath[i]
                ypi = ypath[i]

                if 'x' in self.data[0]:
                    xi = self.data[0]['x'][xpi]
                else:
                    xi = xpi
                if 'y' in self.data[0]:
                    yi = self.data[0]['y'][ypi]
                else:
                    yi = ypi
                
                self.d.append(self.d[-1]+math.sqrt((yi-self.y[-1])**2+(xi-self.x[-1])**2))
                self.z.append(self.data[0]['z'][int(ypi)][int(xpi)])
                self.x.append(xi)
                self.y.append(yi)
                self.xp.append(xpi)
                self.yp.append(ypi)
            

        self.clicks+=1
        
        line_plot.restyle({'x': [self.d], 'y': [self.z]})

    
r = Responder(contour_fig['data'])

In [35]:
g.on_click(r.click)

In [34]:
# CSS styling within IPython notebook - feel free to re-use
from IPython.core.display import HTML
from urllib.request import urlopen

HTML(urlopen(url))

TypeError: HTML expects text, not <http.client.HTTPResponse object at 0x7fb05cd52f98>

In [7]:
from bokeh.models import ColumnDataSource, OpenURL, TapTool
from bokeh.plotting import figure, output_file, show

output_file("openurl.html")

p = figure(plot_width=400, plot_height=400,
           tools="tap", title="Click the Dots")

a = 0.9
colors = [(255, 0, 0), (0, 255, 0), (0, 0, 255), (255, 255, 255), (0, 0, 0), (100, 25, 200), (255, 255, 0), (0, 255, 255), (255, 0, 255), (128, 128, 128), (0, 0, 128), (128, 0, 128)]
sources = ['cnn', 'abc', 'fox', 'nyt', 'reuters', 'wapo', 'huffpo', 'esquire', 'rollingstone', 'cbs', '538', 'washtimes']

import seaborn.apionly as sns
import itertools 
ncolors = 5
palette = sns.palettes.color_palette('colorblind', ncolors)
# as hex is necessary for bokeh to render the colors properly.

for i, color in enumerate(colors):
    print(i)
    source = ColumnDataSource(data=dict(
        x=[1+i, 2+i, 3+i, 4+i, 5+i],
        y=[2, 5, 8, 2, 7],
#         color=["navy", "orange", "olive", "firebrick", "gold"],
        size = [2,4,6,8,10],
        url = [ 'https://fivethirtyeight.com/features/are-moderate-republicans-really-willing-to-kill-the-senate-health-care-bill/',
           'https://fivethirtyeight.com/features/donald-trumps-base-is-shrinking/',
           'https://fivethirtyeight.com/features/jeff-sessions-may-be-tough-on-crime-but-states-want-fewer-prisoners/',
           'https://fivethirtyeight.com/features/this-trump-russia-investigation-could-last-for-years/',
           'https://fivethirtyeight.com/features/we-started-with-14-possible-paths-for-trumps-presidency-which-are-most-likely-now/']

    ))

    p.circle('x', 'y', color=color, alpha=a, size='size', source=source)

    url = "@url"
    taptool = p.select(type=TapTool)
    taptool.callback = OpenURL(url=url)

show(p)

INFO:bokeh.core.state:Session output file 'openurl.html' already exists, will be overwritten.


0
1
2
3
4
5
6
7
8
9
10
11


In [5]:
import seaborn.apionly as sns
import itertools 
ncolors = 5
palette = sns.palettes.color_palette('colorblind', ncolors)
palette

[(0.0, 0.4470588235294118, 0.6980392156862745),
 (0.0, 0.6196078431372549, 0.45098039215686275),
 (0.8352941176470589, 0.3686274509803922, 0.0),
 (0.8, 0.4745098039215686, 0.6549019607843137),
 (0.9411764705882353, 0.8941176470588236, 0.25882352941176473)]

In [8]:
from bokeh.models.widgets import Panel, Tabs
from bokeh.io import output_file, show
from bokeh.plotting import figure

output_file("slider.html")

p1 = figure(plot_width=300, plot_height=300)
p1.circle([1, 2, 3, 4, 5], [6, 7, 2, 4, 5], size=20, color="navy", alpha=0.5)
tab1 = Panel(child=p1, title="circle")

p2 = figure(plot_width=300, plot_height=300)
p2.line([1, 2, 3, 4, 5], [6, 7, 2, 4, 5], line_width=3, color="navy", alpha=0.5)
tab2 = Panel(child=p2, title="line")

tabs = Tabs(tabs=[ tab1, tab2 ])

show(tabs)

In [9]:
tabs

Tabs(id='fca3f744-e7a4-4f77-a59e-1e572abdef29', ...)

In [10]:
[ tab1, tab2 ]

[Panel(id='165fed17-6275-44bd-8036-ecc249e15299', ...),
 Panel(id='95b8fb1c-ff16-4487-9d10-de05baea6cd4', ...)]